In [1]:
import os
from bs4 import BeautifulSoup
from natasha import NamesExtractor  
import json
extractor = NamesExtractor()
import re
#регулярка для имён
fio = re.compile("[А-ЯЁ][а-яё]+ [А-ЯЁ]\.[А-ЯЁ]\.")
fio_cifer = re.compile("[А-ЯЁ][а-яё]+ ФИО[0-9]{1,3}")
fio_short = re.compile("ФИО[0-9]{1,3}")
import pandas as pd

***Эта тетрадка экспортирует только одну функцию get_meta_data, которая принимает текст, а возвращает словарь метаданных(участников судебного процесса)***

### Собираем готовые метаданные

Функция, которая возвращает эксплицитно указанные метаданные и очищенный от тегов текст, а также создаёт словарь, который включает другие метаданные (прокурора, адвоката, секретаря, обвиняемого):

Функция, которая достаёт из строчки русское имя в формате И.И. Иванов:

In [5]:
def get_names(line):
    name = ""
    matches = extractor(line)
    for match in matches:
        fact = match.fact.as_json 
        name = fact["first"]+"."+fact["middle"]+". "+fact["last"].capitalize()
    return name

Функция, которая:
- разбивает текст на три основные части с помощью get_parts
- разбивает вступление либо по запятой (по умолчанию), либо по переносу строки (если запятая не отделяет одни сущности от других)

In [6]:
def splitting_text(text):
    lines = []
    trouble_counter = 0
    splitted_text = text.split("с участием")
    for part in splitted_text:
        lines += [line for line in part.split("\n") if line.strip()]
    for line in lines:
        if len(fio.findall(line)) > 1:
            trouble_counter += 1
    if trouble_counter != 0:
        for part in splitted_text:
            lines = [line for line in part.split(",") if line.strip()]
    return lines       

Функция, которая достаёт строки рядом с нужными нам ключевыми словами, в которых содержатся ФИО:

In [8]:
def meta_lines(text):
    lines = splitting_text(text) 
    add_meta = []
    for i, line in enumerate(lines):  
        if "защитник" in line.lower() or "адвокат" in line.lower(): 
            if fio.findall(line):
                add_meta.append(("advocate", line)) 
            else:
                add_meta.append(("advocate", line+lines[i+1])) 
        if "прокурор" in line.lower() or "обвинител" in line.lower():  
            if fio.findall(line):
                add_meta.append(("prosecutor", line)) 
            else:
                add_meta.append(("prosecutor", line+lines[i+1]))  
        if "секретар" in line.lower():
            if fio.findall(line):
                add_meta.append(("secretary", line)) 
            else:
                add_meta.append(("secretary", line+lines[i+1])) 
        if "подсудим" in line.lower() or "в отношении" in line.lower():
            if fio_cifer.findall(line):
                add_meta.append(("accused", line)) 
            elif fio_short.findall(line):
                add_meta.append(("accused", line)) 
            elif fio.findall(line): 
                add_meta.append(("accused", line)) 
            else:
                if i+1 < len(lines):
                    add_meta.append(("accused", line+lines[i+1])) 
    return add_meta

Функция, которая достаёт из полученных выше строк ФИО формата: ("роль в суде", "ФИО"):

In [9]:
def get_meta_names(meta_lines):
    meta_names = []
    for line in meta_lines:         
        matches = extractor(line[1]) 
        if line[0] == "accused":
            if fio_cifer.findall(line[1]):
                meta_names.append((line[0], fio_cifer.findall(line[1])[0]))
            elif fio_short.findall(line[1]):
                meta_names.append((line[0], fio_short.findall(line[1])[0]))
            elif matches:
                try:
                    for match in matches:
                        fact = match.fact.as_json 
                        meta_names.append((line[0], (fact["last"].capitalize()+" "+fact["first"]+"."+fact["middle"]+".")))
                except KeyError:
                    if fio.findall(line[1]): 
                        meta_names.append((line[0], fio.findall(line[1])[0]))
            elif fio.findall(line[1]): 
                meta_names.append((line[0], fio.findall(line[1])[0]))
        else:
            if matches:
                try:
                    for match in matches:
                        fact = match.fact.as_json 
                        meta_names.append((line[0], (fact["last"].capitalize()+" "+fact["first"]+"."+fact["middle"]+".")))
                except KeyError:
                    if fio.findall(line[1]): 
                        meta_names.append((line[0], fio.findall(line[1])[0]))
            elif fio.findall(line[1]): 
                meta_names.append((line[0], fio.findall(line[1])[0]))
    return meta_names

Функция, которая:
- приводит список с ролями и именами к упорядоченному списку без повторений в нём
- достаёт готовые метаданные с помощью parse_xml в формате словаря
- объединяет те записи из списка, для которых первое значение (роль) совпадают: (адвокат, "Петров А.П.") и (адвокат, "Семёнова Д.В.") приводятся к форме (адвокат, "Петров А.П., Семенова Д.В.")
- добавляет полученные записи из списка в словарь метаданных

In [10]:
def names_to_meta(text):
    meta_data = {}
    updated_names = []
    unique_names = list(set(get_meta_names(meta_lines(text))))
    unique_names.sort()
    #объединение кортежей с одинаковыми нулевыми значениями 
    for i, name in enumerate(unique_names):
        if i+1 < len(unique_names):
            if name[0] == unique_names[i+1][0]:
                updated_names.append((name[0], name[1]+", "+unique_names[i+1][1]))
            elif name[0] != unique_names[i-1][0] or i == 0: 
                updated_names.append((name[0], name[1]))
        elif i+1 >= len(unique_names) and name[0] != unique_names[i-1][0]:
            updated_names.append((name[0], name[1]))
        elif len(unique_names) == 1:
            updated_names.append((name[0], name[1]))

    for name in updated_names:
        meta_data[name[0]] = name[1] 
    return meta_data

Убираем разные падежи для одного имени:

In [13]:
def killing_doubles(text):
    names_dict = names_to_meta(text)
    no_doubles_dict = {}
    for role in names_dict:
        if type(names_dict[role]) == str:
            names_list = names_dict[role].split(",") 
            if "," in names_dict[role]:
                for n, name in enumerate(names_list): 
                    if fio.findall(name): 
                        if n > 0 and name.split()[1] == names_list[n-1].split()[1]  and name.split()[0][:len(name.split()[0])-2] == names_list[n-1].split()[0][0:len(name.split()[0])-2]:
                            names_list.pop(n)
            no_doubles_dict[role] = names_list
        else:
            no_doubles_dict[role] = [names_dict[role]]
    return no_doubles_dict

Финальные метаданные:

In [1]:
def get_meta_data(text):
    dict_with_lists = killing_doubles(text)
    final_meta_dict = {}
    for role in dict_with_lists:
        if len(dict_with_lists[role]) == 1:
            final_meta_dict[role] = dict_with_lists[role][0]
        else:
            final_meta_dict[role] = ",".join(dict_with_lists[role])
    return final_meta_dict

**Весь код ниже работает с путями до файлов и папок, что не относится к извлечению мета-информации из текста**

Функция, которая достаёт ссылки, которые лежат внутри папки (для папки без вложенных папок):

In [105]:
def links_for_folder(path_to_folder):
    paths = []
    for f in os.walk(path_to_folder):
        for path in f[2]:
            paths.append(path_to_folder+path)
    return paths

Функция, которая возвращает словари метаданных для всех документов в папке и также возвращает количество ошибок во время извлечения:

In [106]:
def dicts_for_folder(path_to_folder):
    kerror_counter = 0
    kerror_cases = []
    terror_counter = 0
    terror_cases = []
    folder_dicts = []
    for path in links_for_folder(path_to_folder): 
        try:
            folder_dicts.append(final_meta(path))
        except KeyError: 
            kerror_counter += 1
            kerror_cases.append(path)
            continue
        except TypeError: 
            terror_counter += 1
            terror_cases.append(path)
            continue
    return folder_dicts, (kerror_counter, kerror_cases), (terror_counter, terror_cases)

Функция, которая создаёт для заданной папки датафрейм с метаинформацией:

In [107]:
def df_for_folder(path_to_folder):
    case_list = dicts_for_folder(path_to_folder)[0]
    base_df = pd.DataFrame.from_dict(case_list[0], orient = "index")
    folder_df = base_df.T
    for case in case_list[1:]:
        folder_df.loc[len(folder_df)] = pd.DataFrame.from_dict(case, orient = "index").T.loc[0]   
    return folder_df